In [1]:

import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt 
import random 
import sys
sys.path.append('./lib')
from lib.cleaning import *
from lib.data_prepration import *
from lib.paralellism import *
from lib.mydoc2vec import *
import nltk
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import nltk

from gensim.models import Doc2Vec
from collections import namedtuple
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
#!pip install scipy==1.12
SEED = 448

[nltk_data] Downloading package stopwords to
[nltk_data]     c:\Users\alire\Downloads\neuefische\ds-
[nltk_data]     capstone\.venv\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
file_path = '../data/book_review.csv'
data_prep = DataPreparation(file_path)
data_prep.clean_and_preprocess(True)
data_prep.Normalize()
data_prep.data=data_prep.data[['title', 'price', 'helpfulness', 'score', 'time',
       'summary', 'text', 'description', 'authors', 'publisher',
       'publisheddate', 'categories', 'ratingscount']]
df=data_prep.data
df

,title,price,helpfulness,score,time,summary,text,description,authors,publisher,publisheddate,categories,ratingscount
1,dr. seuss: american icon,21.762656,1.0,5.0,2004-09-21,really enjoyed it,i don't care much for dr. seuss but after read...,philip nel takes a fascinating look into the k...,[Philip Nel],a&c black,2005-01-01,[Biography & Autobiography],272.064662
2,dr. seuss: american icon,21.762656,0.909091,5.0,2004-03-09,essential for every personal and public library,"if people become the books they read and if ""t...",philip nel takes a fascinating look into the k...,[Philip Nel],a&c black,2005-01-01,[Biography & Autobiography],272.064662
3,dr. seuss: american icon,21.762656,1.0,4.0,2004-07-25,phlip nel gives silly seuss a serious treatment,"theodore seuss geisel (1904-1991), aka &quot;d...",philip nel takes a fascinating look into the k...,[Philip Nel],a&c black,2005-01-01,[Biography & Autobiography],272.064662
4,dr. seuss: american icon,21.762656,1.0,4.0,2005-02-10,good academic overview,philip nel - dr. seuss: american iconthis is b...,philip nel takes a fascinating look into the k...,[Philip Nel],a&c black,2005-01-01,[Biography & Autobiography],272.064662
5,dr. seuss: american icon,21.762656,1.0,4.0,2005-09-20,one of america's greatest creative talents,"""dr. seuss: american icon"" by philip nel is a ...",philip nel takes a fascinating look into the k...,[Philip Nel],a&c black,2005-01-01,[Biography & Autobiography],272.064662
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,the idea of history,21.762656,0.736842,4.0,1999-09-18,difficult,"this is an extremely difficult book to digest,...","robin george collingwood, fba (1889 - 1943) wa...",[R. G. Collingwood],white press,2018-06-21,[History],3.000000
2999996,the idea of history,21.762656,1.0,4.0,2012-03-14,quite good and ahead of its time occasionally,this is pretty interesting. collingwood seems ...,"robin george collingwood, fba (1889 - 1943) wa...",[R. G. Collingwood],white press,2018-06-21,[History],3.000000
2999997,the idea of history,21.762656,,4.0,2007-05-27,easier reads of those not well versed in histo...,"this is a good book but very esoteric. ""what i...","robin george collingwood, fba (1889 - 1943) wa...",[R. G. Collingwood],white press,2018-06-21,[History],3.000000
2999998,the idea of history,21.762656,0.090909,5.0,2006-11-09,"yes, it is cheaper than the university bookstore","my daughter, a freshman at indiana university,...","robin george collingwood, fba (1889 - 1943) wa...",[R. G. Collingwood],white press,2018-06-21,[History],3.000000


In [ ]:
df.columns

In [ ]:
print(len(df))
print(df.info())
print(df.describe())

In [ ]:
df.isnull().sum()

In [ ]:
df.score.value_counts()

In [ ]:
data_prep.generate_plots()

In [ ]:
corr_matrix = data_prep.data.select_dtypes(include=[np.number]).corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.show()

In [ ]:
"""!python.exe -m pip install --upgrade pip 
!pip install nltk --upgrade
!pip install gensim --upgrade
!pip install scipy --upgrade"""

In [3]:
join_func = lambda x: ', '.join(map(str, x))
data_prep.data['all']=data_prep.data['title'] + data_prep.data['description'] + data_prep.data['authors'].apply(join_func)+data_prep.data['categories'].apply(join_func)
data_prep.data['all']

1          dr. seuss: american iconphilip nel takes a fas...
2          dr. seuss: american iconphilip nel takes a fas...
3          dr. seuss: american iconphilip nel takes a fas...
4          dr. seuss: american iconphilip nel takes a fas...
5          dr. seuss: american iconphilip nel takes a fas...
                                 ...                        
2999995    the idea of historyrobin george collingwood, f...
2999996    the idea of historyrobin george collingwood, f...
2999997    the idea of historyrobin george collingwood, f...
2999998    the idea of historyrobin george collingwood, f...
2999999    the idea of historyrobin george collingwood, f...
Name: all, Length: 2359775, dtype: object

In [ ]:

#tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data_prep.data['all'])]


In [4]:

pls=Parallelism(8)
pls.set_worker()

def preprocess_document(text):
  import nltk
  from nltk.tokenize import word_tokenize
  from nltk.corpus import stopwords
  tokens = word_tokenize(text.lower())
  cleaned_tokens = [word for word in tokens if word not in stopwords.words('english')]
  return cleaned_tokens
#preprocessed_documents = pls.do_paralell(data_prep.data,'all',preprocess_document)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [5]:
_doc2vec = Doc2VecRecommender()
_doc2vec.train(data_prep.data)

In [ ]:
_doc2vec.model.save("doc2vec_model_hole")


In [ ]:
_doc2vec_load = Doc2VecRecommender(data_prep.data)
_doc2vec_load.load_model('doc2vec_model_hole')
search='after ice age a lion king is back'

In [ ]:
_doc2vec_load.recommend_by_text(search) 